In [ ]:
# !wget http://nlp.stanford.edu/data/glove.42B.300d.zip
# !wget https://s3.ap-south-1.amazonaws.com/discovery-nlp-data/dataset.csv

In [18]:
import pandas as pd
import numpy as np
import re
df = pd.read_csv('/home/yash/Downloads/dataset.csv',sep=",",header=None ,names=['sentence','type'])

print df.head()
# df.apply(str)
# type(df['sentence'][0])
# embedding_matrix=np.load("my_matrix.dat")

                                            sentence   type
0  MUCH better than the West Hollywood property w...  Hotel
1      Stay away from room service in my opinion....  Hotel
2                           Room service was superb.  Hotel
3  Stayed in a king suite for 11 nights and yes i...  Hotel
4  The location close to the 72nd Street subway s...  Hotel


In [19]:
Y = df['type']
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

le = preprocessing.LabelEncoder()
le.fit(Y)
Y=le.transform(Y) 
labels = to_categorical(np.asarray(Y))

In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from unidecode import unidecode
# fit_on_texts(textdata))
MAX_NB_WORDS = 16384
MAX_SEQUENCE_LENGTH=128


In [29]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, split=' ')

alist=[str(x) for x in df['sentence'].values ]
tokenizer.fit_on_texts(alist)
X=tokenizer.texts_to_sequences(alist)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size = 0.2)


In [30]:
embeddings_index = {}

f = open('/home/yash/glove.42B.300d.txt')
for line in f:
    #print line
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

emb_dimension=300
embedding_matrix = np.zeros((len(word_index) + 1, emb_dimension))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        


Found 1917494 word vectors.
Found 3650 unique tokens.


In [31]:
# numpy.save(, embedding_matrix)
embedding_matrix.dump('embd.dat')

In [32]:
# from keras.layers import Embedding
# word_index_len=20465
# emb_dimension=300
# embedding_layer = Embedding(word_index_len,
#                             emb_dimension,
#                             weights=[embedding_matrix],
#                             input_length=MAX_SEQUENCE_LENGTH,
#                             trainable=False)

In [36]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.callbacks import History ,ReduceLROnPlateau
from keras import regularizers
from keras import optimizers
history = History()
model = Sequential()
model.add(Embedding(len(word_index)+1,
                            300,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False))
# model.add(embedding_layer)

model.add(LSTM(256,
               return_sequences=True, 
               dropout=0.4, 
               recurrent_dropout=0.4, 
               activation='relu',
               kernel_regularizer=regularizers.l2(0.001)))#,activity_regularizer=regularizers.l1(0.01)))

model.add(LSTM(128, 
               dropout=0.4, 
               recurrent_dropout=0.4, 
               activation='relu',
               kernel_regularizer=regularizers.l2(0.001)))#,activity_regularizer=regularizers.l1(0.01)))

#model.add(LSTM(256, dropout_U=0.5, dropout_W=0.5, activation='relu', kernel_regularizer=l2_reg))
model.add(Dense(4,activation='softmax'))#,kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)))
rmsprop = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999)
model.compile(loss = 'categorical_crossentropy', optimizer=rmsprop,metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 128, 300)          1095300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128, 256)          570368    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
Total params: 1,863,304
Trainable params: 768,004
Non-trainable params: 1,095,300
_________________________________________________________________


In [37]:

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=5, min_lr=0.0001, cooldown=5)
history=model.fit(x_train, y_train,
          batch_size=128,
          epochs=2,
          verbose=1,
          callbacks=[reduce_lr],
          validation_data=(x_test, y_test))

Train on 1575 samples, validate on 394 samples
Epoch 1/2
1575/1575 [==============================] - 8s 5ms/step - loss: 2.1614 - acc: 0.3778 - val_loss: 1.6609 - val_acc: 0.5431
Epoch 2/2
1575/1575 [==============================] - 3s 2ms/step - loss: 1.5773 - acc: 0.5276 - val_loss: 1.6266 - val_acc: 0.5660


In [39]:
example = tokenizer.texts_to_sequences(["food was worse"])
example = pad_sequences(example, maxlen=MAX_SEQUENCE_LENGTH)
print model.predict(example),type(model.predict(example))
le.inverse_transform(np.argmax(model.predict(example)))

[[0.6241612  0.17492281 0.08832416 0.11259187]] <type 'numpy.ndarray'>


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


'FB'

In [ ]:
# evaluate the model of test data

from sklearn.metrics import classification_report
predictions = model.predict(x_test)
#print(classification_report(predictions,y_test))
print("Accuracy :",model.evaluate(x_test,y_test))